In [39]:
# Read the input file and print the length of the text
with open('/Users/manpreet.singh/git/gpt/notebooks/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

1115394


In [40]:
# Create the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [41]:
# Create the character to index and index to character mapping - tokenization
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [42]:
# create the tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [43]:
# lets split into training and validation set
n = int(0.9 * len(data)) # 90% training, 10% validation
train_data, valid_data = data[:n], data[n:]

In [44]:
# This will help transformer see the context ranging from single character to the block_size
# This is the time dimension for the transformer
# Chunks and block size
block_size = 8 # context length
train_data[:block_size + 1] # basically 8 examples are packed in block_size

x = train_data[:block_size]
y = train_data[:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} then target: {target}')


When input is tensor([18]) then target: 18
When input is tensor([18, 47]) then target: 47
When input is tensor([18, 47, 56]) then target: 56
When input is tensor([18, 47, 56, 57]) then target: 57
When input is tensor([18, 47, 56, 57, 58]) then target: 58
When input is tensor([18, 47, 56, 57, 58,  1]) then target: 1
When input is tensor([18, 47, 56, 57, 58,  1, 15]) then target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target: 47


In [45]:
# Lets us get both time and batch dimension done
# Multiple sequences are selected in a batch, each sequence is block_size long, gpus are good at parallel processing
torch.manual_seed(1337)
batch_size = 4 # number of sequences in a mini batch
block_size = 8 # context length of each sequence

def get_batch(split):
    # generate a small batch of data of input x and target y
    if split == 'train':
        data = train_data
    else:
        data = valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # starting index of each sequence
    x = [data[i:i+block_size] for i in ix] # get the sequences
    y = [data[i+1:i+block_size+1] for i in ix] # get the target sequences
    return torch.stack(x), torch.stack(y) # stack the sequences

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('-----------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'Batch: {b}, When input is {context} then target: {target}')
    


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
Batch: 0, When input is tensor([24]) then target: 43
Batch: 0, When input is tensor([24, 43]) then target: 58
Batch: 0, When input is tensor([24, 43, 58]) then target: 5
Batch: 0, When input is tensor([24, 43, 58,  5]) then target: 57
Batch: 0, When input is tensor([24, 43, 58,  5, 57]) then target: 1
Batch: 0, When input is tensor([24, 43, 58,  5, 57,  1]) then target: 46
Batch: 0, When input is tensor([24, 43, 58,  5, 57,  1, 46]) then target: 43
Batch: 0, When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) then target: 39
Batch: 1, When input is tensor([44]) then target: 53
Batc

In [46]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [49]:
# simple Bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads of the logits for the next token from the look up table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx is the input to the model
        logits = self.token_embedding_table(idx) # (batch_size, block_size, vocab_size) (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is the input to the model (B*T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # (B*T, C)
            logits = logits[:, -1, :] # get the logits for the last token (B, C), we use only the last token to predict the next token
            probs = F.softmax(logits, dim=-1) # (B, C)
            next_token = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat([idx, next_token], dim=1) # (B, T+1)
            #print(idx)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [50]:
for name, param in model.named_parameters():
    print(name, param.shape)

import torch.optim as optim
# Create a pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

token_embedding_table.weight torch.Size([65, 65])


In [51]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # forward pass
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3796486854553223


In [52]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


llo br. ave aviasurf my, mayo t ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whe, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
By bre ndy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n sar; my w, fredeeyong
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, pannd e Yolde Th li


In [ ]:
# so far we see that the above model is a very simple one and looking at the last token to predict the next token,
# not considering the context, i.e. no token is talking to each other.

# study the embedding class

SUPER CRITICAL

In [80]:
# Mathematical trick for self attention

# toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch size, time dimension, channels
x = torch.randn(B, T, C) # input to the transformer
print(x.shape)

# 8 tokens above have not been talking to each other at all, so far
# token at 5th position should not talk to future tokens, only to the past tokens
# C is also called the information at a particular token

# one of the best way for getting the information from the past tokens is to take the weighted average of the past tokens, 
# like for the 5th token, we take the weighted average of the first 4 tokens + information at the 5th token
# this average becomes like the feature vector for the 5th token

# using average is not a good idea, its a weak form of information carrying and very lossy
# spatial arrangments of the tokens is not considered in the above approach, very bad :)

# lets write the above, we want x[b, t] = mean_{i<t} x[b, i]
# version 1
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0) # (C)

# what we did above is created xbag of words, xbow - to store the information of the past tokens (average of the past tokens)

torch.Size([4, 8, 2])


In [81]:
# version 2
# lets do the same using matrix multiplication
wei = torch.tril(torch.ones(T, T)) # (T, T) lower triangular matrix
wei = wei / wei.sum(1, keepdim=True) # (T, T) row sum to 1
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow2, xbow)

True

In [82]:
# version 3
# lets do the same using softmax
tril = torch.tril(torch.ones(T, T)) # (T, T) lower triangular matrix
wei = torch.zeros(T, T) # (T, T)
wei = wei.masked_fill(tril == 0, float('-inf')) # (T, T) mask the upper triangular part
wei = F.softmax(wei, dim=-1) # (T, T) row sum to 1
xbow3 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow3, xbow)

# something very interesting is happening here, 
# we create a tril matrix,
# we create a wei matrix with all 0s, then we mask the upper triangular part with -inf (inf to make sure that you cannot interact with the future tokens)
# then we apply softmax to the masked matrix,
# then we multiply the softmax matrix with the input x
# this gives us the xbow, the information of the past tokens
# **** the wei matrix starts with zero and then starts storing the effinities of different tokens with respect to the current token ****
# **** the effinities are calculated using the softmax function, taking exp and then div by the sum ****

True

In [91]:
# version 4 - self attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch size, time dimension, channels
x = torch.randn(B, T, C) # input to the transformer

# lets see a single head perform self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # (C, H)
query = nn.Linear(C, head_size, bias=False) # (C, H)
value = nn.Linear(C, head_size, bias=False) # (C, H)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
# At this point, at each position, for each token we have a key and a query. All done in parallel. None is yet communicating with each other.
# lets comminicate
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T), for every row in B, the effinities are given by a square matrix T x T
print(wei[0])

tril = torch.tril(torch.ones(T, T)) # (T, T) lower triangular matrix
# wei = torch.zeros((T, T)) # (T, T)
wei = wei.masked_fill(tril == 0, float('-inf')) # (T, T) mask the upper triangular part
print('-----------------')
print(wei[0])
wei = F.softmax(wei, dim=-1) # (T, T) row sum to 1
print('-----------------')
print(wei[0])

v = value(x) # (B, T, 16)
out = wei @ v # (B, T, T) @ (B, T, 16) -> (B, T, 16)
#out = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
out.shape
print('-----------------')
# print(out[0])


# above is a simple mechanism of getting information from the past tokens
# but what we want is that each token wants to talk to all prev tokens or rather look for data dependencies in the past tokens, 
# vowels are interested in knowing about consonants in the past, consonants are interested in knowing about vowels in the past

# [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]]
# 8th token knows what content it has and its position. Based on which the token creates a query (i am looking for that), i am a vowel and i am looking for consonants
# upto positions 1 to 7. All the token from 1 to 7 emits keys, and may be one of the channel indicates that it is consonant. When query and key are dot prodcuted, it creates
# a high value indicating the higher effinity.

# at the end we dont aggregate with x exactly, but with value v.
# x is the private information of the token, v is more like a public information of the token
# i am 5th token, my original identity is kept in x, 
# v instead has, for a single head, this is what i am interested in, here is what i have, if u find me inetresting here is what i will communicate to you

# 1. ultimately attention is just a communication mechanism. It is a way to communicate between the tokens.
# 2. attention mechanism in its basic form do not have idea about position (notion of space should be added), unlike convolutional neural networks
# 3. across batch dimension, we are not communicating, we are communicating only within the batch dimension
# 4. sometimes like for sentiment analysis, all tokens can communicate with each other, basically we call it encoder block, basically we just remove line with '-inf'
# 5. above implementation is called decoder block (nodes from future never talk to the past nodes), this is a.k.a autoregressive model
# 6. unlike self attention above where k,q,v are working on same node pool. The cross attention can have q from one node pool and k,v from another node pool, q is pulling information from second node pool
# 7. in the main attention formula, we also have to divide by sqrt(d_k). d_k is the head size, also called scaled attention. 
#    k, q are unit variance i.e. unit guassian inputs, 'wei' will be unit variance too. see below.
#    'wei' is fed to softmax, its important to variance of wei near to 1. If variance is high, softmax will saturate and if variance is low, softmax will be very peaky.


tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)
-----------------
tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496, 

In [95]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

# wei's variance is in the order of head_size, so we divide by sqrt(head_size) to make it unit variance
print(k.var(), q.var(), wei.var())

tensor(1.0104) tensor(1.0204) tensor(1.1053)


In [84]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [58]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [59]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [60]:
# you see above how the element wise average is taken for the past tokens
# e.g. [-0.0341,  0.1332] is the average of all the previous tokens

# Very inefficient way of doing the above, lets apply the trick

In [64]:
# lets do the above using matrix multiplication

# toy example
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

# here row of a is multiplied with the column of b, dot product is taken

# tril is the lower triangular matrix
torch.tril(torch.ones(3, 3))
# we used tril in a, the result is amazing, a @ b, the result is the weighted average
# imagine if we apply tril matrix and do matrix/torch.sum, multiply with x, we will get the weighted average of the past tokens


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])